In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [1]:
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
import pandas as pd


df = pd.read_csv('../input/training.1600000.processed.noemoticon.csv',engine='python',header=None)

In [3]:
df.columns=['target','id','date','flag','user','text']
df = df.sample(frac=1).reset_index(drop=True)

In [4]:
df_n = pd.get_dummies(df['target'])


In [5]:
df=df.drop(axis=1,labels='target')


In [6]:
df['positive']=df_n[4]



In [7]:
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
corpus=[]
ps=SnowballStemmer('english')
for i in range(df.shape[0]):
  review=re.sub('[^a-zA-Z]',' ',df['text'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  review=' '.join(review)
  corpus.append(review)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(max_features=650)


In [9]:
x=v.fit_transform(corpus).toarray()

In [11]:
x_train=x[:1300000,:]
y_train=df.iloc[:1300000,-1].values
x_test=x[130000:,:]
y_test=df.iloc[130000:,-1].values


In [26]:
from keras.models import Sequential
from keras.layers import Dense

In [34]:
classifier=Sequential()

In [35]:
classifier.add(Dense(output_dim=300,init='uniform',activation='relu',input_shape=(650,)))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_shape=(650,), units=300, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.


In [36]:
classifier.add(Dense(output_dim=200,init='uniform',activation='relu'))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=200, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.


In [37]:
classifier.add(Dense(output_dim=100,init='uniform',activation='relu'))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.


In [38]:
classifier.add(Dense(output_dim=20,init='uniform',activation='relu'))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=20, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.


In [39]:
classifier.add(Dense(output_dim=1,init='uniform',activation='sigmoid'))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.


In [40]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [41]:
classifier.fit(x_train,y_train,batch_size=32500,nb_epoch=35)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/35
1300000/1300000 [==============================] - 33s 25us/step - loss: 0.6871 - acc: 0.6303
Epoch 2/35
1300000/1300000 [==============================] - 32s 25us/step - loss: 0.5410 - acc: 0.7307
Epoch 3/35
1300000/1300000 [==============================] - 33s 25us/step - loss: 0.5227 - acc: 0.7347
Epoch 4/35
1300000/1300000 [==============================] - 33s 25us/step - loss: 0.5207 - acc: 0.7359
Epoch 5/35
1300000/1300000 [==============================] - 33s 25us/step - loss: 0.5182 - acc: 0.7373
Epoch 6/35
1300000/1300000 [==============================] - 33s 25us/step - loss: 0.5145 - acc: 0.7399
Epoch 7/35
1300000/1300000 [==============================] - 33s 25us/step - loss: 0.5106 - acc: 0.7426
Epoch 8/35
1300000/1300000 [==============================] - 33s 25us/step - loss: 0.5062 - acc: 0.7456
Epoch 9/35
1300000/1300000 [==============================] - 33s 25us/step - loss: 0.5022 - acc: 0.7482
Epoch 10/35
1300000/1300000 [==========================

In [42]:
y_pred=classifier.predict(x_test)

In [43]:
pred=(y_pred>0.5)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test, pred))
print('\n')
print(classification_report(y_test, pred))